In [1]:
%matplotlib inline

from io import StringIO
from PIL import Image
import imageio
import numpy as np
import pylab
import matplotlib.pyplot as plt
import os

In [2]:
homer_vid = "Documents/multimedia-systems-spring-2018/resources/homer.mp4"
resources_path = "Documents/multimedia-systems-spring-2018/resources"

In [18]:
def QuantizeVid(frames, Q=10):
    frames_q = []
    for frame in frames:
        frames_q.append(np.around(np.around(frame/Q, 2)*Q, 2))
    return frames_q

In [19]:
def QuantizeImage(img, Q=10):
    return np.around(np.around(img/Q, 2)*Q, 2)

In [5]:
def VidConvertGrayScale(rgbVid):
    '''Converts numpy array vid to BW vid'''
    frames_bw = []
    for frame in rgbVid:
        frames_bw.append(np.dot(frame[:,:,:3], [0.2989, 0.587, 0.114]))
    return np.array(frames_bw)

In [6]:
def parse_vid(filename):
    vid = imageio.get_reader(filename,  'ffmpeg')
    fps = vid.get_meta_data()['fps']
    num = 0
    frames = []
    while 1:
        try:
            image = vid.get_data(num)
            frames.append(image)
            num+=1
        except IndexError:
            break

    return np.array(frames), fps

In [7]:
def calc_diff(frames):
    dif_lst = []
    for i in range(1, frames.shape[0]):
        dif_lst.append(frames[i-1,:,:]-frames[i,:,:])
    return np.array(dif_lst)

In [8]:
def comp_vid(frames):
    enc_list= []
    for frame in frames:
        enc_list.append(comp_image(frame))
    return np.array(enc_list)

In [9]:
def comp_image(frame):
    cnt = 0
    curr_el = frame[0,0]

    encoded = []

    for i in range(frame.shape[0]):
        for j in range(frame.shape[1]):
            if frame[i,j] == curr_el:
                cnt += 1
            else:
                encoded.append('{0}!{1}'.format(curr_el, cnt))
                curr_el = frame[i, j]
                cnt = 1        

    encoded.append('{0}!{1}'.format(curr_el, cnt))
    return np.array(encoded)

In [10]:
def vid_encode(filename):
    # parse RGB video
    frames, fps = parse_vid(filename)
    # convert to BW
    bw_frames = VidConvertGrayScale(frames)
    # calculate the frame difference matrix
    diffs = calc_diff(bw_frames)
    # quantize the matrix
    q_diffs = QuantizeVid(diffs, Q=100)
    # compress quantized matrix and 1st frame
    q_diffs_comp = comp_vid(q_diffs)
    img1_q = QuantizeImage(bw_frames[0,:,:])
    first_img_comp = comp_image(img1_q)
    
    return first_img_comp, q_diffs_comp, bw_frames[0,:,:].shape, fps


In [11]:
def decode_vid(frames, dims):
    dec_list = []
    for frame in frames:    
        original_dimensions = dims
        decoded = []

        for i in frame:
            symbol, count = i.split('!')
            decoded.extend([float(symbol)]*int(count))   

        dec = np.array(decoded, dtype=np.float64).reshape(original_dimensions)
        dec_list.append(dec)
    return np.array(dec_list)

In [12]:
def decode_image(frame, dims):
    original_dimensions = dims
    decoded = []

    for i in frame:
        symbol, count = i.split('!')
        decoded.extend([float(symbol)]*int(count))   

    decoded = np.array(decoded, dtype=np.float64).reshape(original_dimensions)
    return np.array(decoded)

In [13]:
def reconstruct_vid(img1, diffs):
    final_vid  = [img1]
    cnt = 1
    for frame in diffs:
        final_vid.append(final_vid[-1]-frame)
        
    
    return np.array(final_vid)

In [14]:
def save_vid(frames, path, name, fps):
    writer = imageio.get_writer(os.path.join(path,name), fps=fps,mode="I")

    for frame in frames:
        writer.append_data(frame)
    writer.close()

In [27]:
import pickle

In [28]:
first_img_comp , q_diffs_comp, dims, fps = vid_encode(homer_vid)

pickle.dump(os.path.join(resources_path,'img1.pckl'), first_img_comp)
pickle.dump(os.path.join(resources_path,'qdiffs.pckl'), q_diffs_comp)

TypeError: file must have a 'write' attribute

In [21]:
img1 = decode_image(first_img_comp, dims)
diffs = decode_vid(q_diffs_comp, dims)
tmp2 = reconstruct_vid(img1, diffs)

save_vid(tmp2,resources_path,'homer_bw_final_10.mp4', fps)

C:\Users\theotheo\Anaconda3\lib\site-packages\imageio\core\util.py:104: UserWarning: Conversion from float64 to uint8, range [26.2, 242.2]
  'range [{2}, {3}]'.format(dtype_str, out_type.__name__, mi, ma))
C:\Users\theotheo\Anaconda3\lib\site-packages\imageio\core\util.py:104: UserWarning: Conversion from float64 to uint8, range [16.0, 249.6]
  'range [{2}, {3}]'.format(dtype_str, out_type.__name__, mi, ma))
C:\Users\theotheo\Anaconda3\lib\site-packages\imageio\core\util.py:104: UserWarning: Conversion from float64 to uint8, range [21.30000000000001, 248.0]
  'range [{2}, {3}]'.format(dtype_str, out_type.__name__, mi, ma))
C:\Users\theotheo\Anaconda3\lib\site-packages\imageio\core\util.py:104: UserWarning: Conversion from float64 to uint8, range [22.5, 250.8]
  'range [{2}, {3}]'.format(dtype_str, out_type.__name__, mi, ma))
C:\Users\theotheo\Anaconda3\lib\site-packages\imageio\core\util.py:104: UserWarning: Conversion from float64 to uint8, range [21.69999999999999, 247.5]
  'range [{